In [0]:
import time
import numpy as np
import tensorflow as tf

##################################################
# Imports the necessary modules from TF-graphics #
##################################################
from colabtools import adhoc_import
with adhoc_import.Google3():
  from tensorflow_graphics.transformation import quaternion

####################
# Helper functions #
####################
# reset graph
tf.reset_default_graph()

# Sample 3D points
nb_points = 40
points = tf.placeholder(dtype=tf.float32, shape=(nb_points, 3))

# Forms a random translation
with tf.name_scope("translation_variable"):
  random_translation_base = tf.Variable(
      np.random.uniform(-1.0, 1.0, (3,)), dtype=tf.float32)
  random_translation = tf.tile(random_translation_base, [nb_points])
  random_translation = tf.reshape(random_translation, [nb_points, 3])

# Forms a random quaternion
hi = np.pi
lo = -hi
random_angles = np.random.uniform(lo, hi, (3,)).astype(np.float32)
with tf.name_scope("rotation_variable"):
  random_quaternion_base = tf.Variable(quaternion.from_euler(random_angles))
  random_quaternion = tf.manip.tile(random_quaternion_base, [nb_points])
  random_quaternion = tf.reshape(random_quaternion, [nb_points, 4])

# Transforms the sampled point using the quaternion and translation
with tf.name_scope("rigid_transformation"):
  transformed_points = quaternion.rotate(points,
                                         random_quaternion) + random_translation

# Sets the optimization problem
with tf.name_scope("loss"):
  loss = tf.nn.l2_loss(points - transformed_points) / nb_points

learning_rate = 0.2
with tf.name_scope("optimization"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

with tf.name_scope("initialize_variables"):
  init = tf.initialize_all_variables()

In [0]:
#############
# 3D Viewer #
#############
from colabtools.labs import threejs

threejs_cell = threejs.ThreeJsCell(height=600, width=600)

# Points and colors
random_points = np.random.uniform(-1.0, 1.0, (nb_points, 3))
blue_color = np.tile((0.0, 0.4, 0.8), (nb_points, 1))
green_color = np.tile((0.0, 0.8, 0.4), (nb_points, 1))

# Adds target points
target_points = threejs.Points(
    random_points.tolist(),
    colors=blue_color.tolist(),
    size=0.1,
    opacity=0.8,
    transparent=True)
threejs_cell.Add(target_points)
# Adds the current 'solution' points.
source_points = threejs.Points(
    np.zeros((nb_points, 3)).tolist(),
    colors=green_color.tolist(),
    size=0.1,
    opacity=0.8,
    transparent=True)

threejs_cell.Add(source_points)
# Add frame
frame_points = np.array([[0., 0., 0.], [1., 0., 0.], [0., 0., 0.], [0., 1., 0.],
                         [0., 0., 0.], [0., 0., 1.]])
frame_colors = np.array([[0.8, 0.2, 0.2], [0.8, 0.2, 0.2], [0.2, 0.8, 0.2],
                         [0.2, 0.8, 0.2], [0.2, 0.2, 0.8], [0.2, 0.2, 0.8]])
lines = threejs.LinePieces(frame_points.tolist(), colors=frame_colors.tolist())
threejs_cell.Add(lines)

# Camera
threejs_cell.BuildCamera(position=(0, 0, 3.0))
threejs_cell.AddTrackballControls()
threejs_cell.LookAt(0.0, 0.0, 0.0)

threejs_cell.Animate()

################
# Optimization #
################
session = tf.Session()
session.run(init)
nb_iterations = 50
for it in range(nb_iterations):
  _, loss_v, transformed_points_v = session.run(
      [training_op, loss, transformed_points], {points: random_points})
  source_points.geometry.vertices = threejs.Vector3Array(
      transformed_points_v.tolist())
  source_points.geometry.verticesNeedUpdate = True

  time.sleep(0.1)